# Hugging Face Library 'Transformer'およびT5Tokenizerのダウンロード

参考(https://qiita.com/takubb/items/fd972f0ac3dba909c293)これを基に改造し、最新のGoogle Colaboratoryで動作するようにした

In [1]:
!export CUDA_LAUNCH_BLOCKING=1
# !pip install torch
#!pip install torchvision
#!pip install transformers
#!apt install swig
# Sentencepieceのインストール
#!pip install sentencepiece
#!pip install mecab-python3
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-roberta-base")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'
import gzip
import shutil
import sqlite3
import random
from math import ceil
import re
import csv
import glob
import torchvision
import statistics
import numpy as np
import lightgbm
import pickle
import statistics
import MeCab
import re
import copy
import scipy.stats as stats

import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

# PyTorchとGPU設定

In [3]:
#!pip install torch
import torch
# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# Data Augmentation kansuu

In [4]:
# synreplace - replace kasho kosuu
# randinsert - tasu kotoba no kazu
# randdelete - delete kakuritsu
# randswap - swap kaisuu

class synreplace(object):
    def __init__(self, num):
        self.num = num
        self.model = RobertaForMaskedLM.from_pretrained("rinna/japanese-roberta-base")
    def __call__(self, textlist):
        # textlist: honbun no list
        textlen = torch.where(textlist == 3)[0][0]
        for n in range(self.num):
            # chikan shiro
            masked_idx = random.randint(2, textlen-1)
            textlist[masked_idx] = 6
            # convert to tensor
            token_tensor = torch.tensor(textlist)
            # get the top 10 predictions of the masked token
            self.model = self.model.eval()
            with torch.no_grad():
                outputs = self.model(torch.unsqueeze(token_tensor, 0))
                predictions = outputs[0][0, masked_idx].topk(1)
            for i, index_t in enumerate(predictions.indices):
                index = index_t.item()
            textlist[masked_idx] = index
        return textlist

class randinsert(object):
    def __init__(self, num):
        self.num = num
    def __call__(self, textlist):
        for n in range(self.num):
            insword = textlist[random.randint(1,len(textlist)-1)]
            i = random.randint(1,len(textlist)-1)
#            print('len: ', len(textlist))
#            print(i)
            while textlist[i] == 3:
                i = random.randint(1,len(textlist)-1)
#                print(i)
            textlist = torch.cat([textlist[0:i], torch.tensor([insword]), textlist[i:-1]])
        return textlist

class randdelete(object):
    def __init__(self, num):
        self.num = num
    def __call__(self, textlist):
#        print(textlist.shape)
        for i in range(3,len(textlist)-1):
            if textlist[i] == 3:
                continue
            r = random.uniform(0, 1)
            if r < self.num:
#                textlist.pop(i)
                textlist = torch.cat([textlist[0:i], textlist[i+1:], torch.tensor([3])])
#                print(textlist)
        return textlist

class randswap(object):
    def __init__(self, num):
        self.num = num
    def __call__(self, textlist):
        counter = 0
        #rs_sents = np.zeros(len(textlist), dtype=object)
        for i in range(len(textlist)):
            while self.num > counter:
                box = 0
                random_idx_1 = random.randint(1, len(textlist)-1)
                while textlist[random_idx_1] == 3:
                    random_idx_1 = random.randint(0, len(textlist)-1)
                random_idx_2 = random.randint(1, len(textlist)-1)
                while random_idx_1 == random_idx_2 or textlist[random_idx_2] == 3:
                    random_idx_2 = random.randint(0, len(textlist)-1)
                    # print(random_idx_1, random_idx_2)
                box = textlist[random_idx_1]
                textlist[random_idx_1] = textlist[random_idx_2]
                textlist[random_idx_2] = box
                counter += 1
        return textlist

# Custom Tensor Dataset
https://stackoverflow.com/questions/55588201/pytorch-transforms-on-tensordataset

# データセットの準備

ライブドアニュースコーパスをダウンロード

    ダウンロードしたファイルは圧縮（tar.gz形式）ファイル
    様々なジャンル（IT,スポーツ,家電,映画など）のWEBメディアごとにフォルダに記事がテキストファイルで保存されている
    
以下、ファイルを読み込んで、必要な部分を抽出

In [5]:
#urllib.request.urlretrieve("https://www.rondhuit.com/download/ldcc-20140209.tar.gz", "ldcc-20140209.tar.gz")
# ダウンロードした圧縮ファイルのパスを設定
#tgz_fname = "ldcc-20140209.tar.gz" 
# 2つをニュースメディアのジャンルを選定
mydata = '/export/livedoor' 
#処理をした結果を保存するファイル名 
tsv_fname = "all_text.tsv" 

def remove_brackets(inp):
    output = re.sub(u'[〃-〿]', '',(re.sub('＝|=|×|\(|\)|“|”|（|）|／|\[|\]| |　|…|・|\n|\t|/|＜|＞|@|＠', '', re.sub(u'[ℊ-⿻]', '', inp)))) #210A ~ 2FFF
    return output

"""
def read_url(f):
    url = next(f)
    return url[:-1]

def read_date(f):
    date = next(f)
    date = remove_brackets(date.encode().decode('utf-8'))
    return date[:-1]
"""

def read_title(f):
    next(f)
    next(f)
    title = next(f)
    title = remove_brackets(title.encode().decode('utf-8'))
    return title[:-1]

def read_para(f):
    p = ''
    while True:
        try:
            para = next(f)
            para = remove_brackets(para.encode().decode('utf-8'))
            p += para
        except StopIteration:
            break
    return p [:-1]

In [6]:
#directory = ['/export/livedoor/dokujo-tsushin', '/export/livedoor/it-life-hack']
#target_genre = ["dokujo-tsushin", "it-life-hack"] 
directory = ['/export/livedoor/dokujo-tsushin', '/export/livedoor/peachy']
target_genre = ["dokujo-tsushin", "peachy"] 
zero_fnames = []
one_fnames = []

if os.path.exists(tsv_fname) == True:
    with open(tsv_fname, "r+") as f:
        f.truncate(0)

for i in range(2):
    for filename in os.listdir(directory[i]):
        if "LICENSE.txt" in filename:
            continue
        f = os.path.join(directory[i], filename)
        #if os.path.isfile(f):
        #    print(f)
        if target_genre[0] in f and f.endswith(".txt"):
            with open(tsv_fname, "a") as wf:
                writer = csv.writer(wf, delimiter='\t')
                with open(f) as zf:
                    title = read_title(zf)
                    para = read_para(zf)
                    row = [target_genre[0], '0', title, para]
                    writer.writerow(row)
            continue
        if target_genre[1] in f and f.endswith(".txt"):
            with open(tsv_fname, "a") as wf:
                writer = csv.writer(wf, delimiter='\t')
                with open(f) as zf:
                    title = read_title(zf)
                    para = read_para(zf)
                    row = [target_genre[1], '1', title, para]
                    writer.writerow(row)
            continue

pandasでデータを読み込み

In [7]:
import pandas as pd
# データの読み込み
df = pd.read_csv("all_text.tsv", 
                 delimiter='\t', header=None, names=['media_name', 'label','title','sentence'])


#df_ = pd.read_csv("summary_set_dokujo_it.tsv", 
#                 delimiter='\t', header=None, names=['summaries'])

df_ = pd.read_csv("summary_set_dokujo_peachy.tsv", 
                 delimiter='\t', header=None, names=['summaries'])

# データの確認
print(f'データサイズ： {df.shape}')
df_.sample(10)

データサイズ： (1712, 4)


,summaries
880,日本では職場で働き続ける人の数が急増している。労働力調査によると、女性の約10人に1人が働き...
91,春が近づき、東京オリンピック(五輪)およびパラリンピック大会が始まって以降、女性の収入が激減...
1467,バレンタインデーは、世界中の恋人にまたもチョコレートを与えてくれるかもしれない。
1087,メッセージ・メッセージ ・ あなたは自分の誕生日に幸せを願っているだろうか?
807,たくさんの女性が、自分たちの誕生日を祝おうと、挙句に応じないままメッセージを送った。しかし、...
1302,レイト夏が近づき、新しいサービスが発表され、人気を集めている日本航空(JTB)は、今夏にハワ...
998,新型コロナウイルスのパンデミックによる感染症「COVID-19」が世界中で広がっている。こう...
448,あるお笑い芸人が、自分を成長させるのはどういうことか。
582,小鳥居yukiの「テンパリスト」は、第2次世界大戦中に100万部以上を突破した。
148,ある若い女性の恋人が、半同棲中だったという疑惑が浮き彫りになっている。


//文章データをsentences、ラベルデータを labelsに保存、以降この2変数だけを利用

In [8]:
mn = df.media_name.values
labels = df.label.values
titles = df.title.values
sentences = df.sentence.values
summaries = df_.summaries.values

In [9]:
len(summaries)

1712

In [10]:
print(type(labels))

<class 'numpy.ndarray'>


In [11]:
tagger = MeCab.Tagger("-Owakati")

def make_wakati(sentence):
  # MeCabで分かち書きを行う
    sentence = tagger.parse(sentence)
  # 半角全角英数字などは削除する
#    sentence = re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+', " ", sentence)
  # 記号なども削除する
#    sentence = re.sub(r'[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=)／*&^%$#@!~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+', "", sentence)
  # スペース区切で形態素の配列に変換する
    wakati = sentence.split(" ")
  # 空要素を削除する
    wakati = list(filter(("").__ne__, wakati))
    return wakati

In [12]:
hoge=make_wakati(sentences[2])
hoge[0]

'芸人'

In [13]:
wakati_sentences = []

for i in range(len(sentences)):
    wakati_sentences.append(make_wakati(sentences[i]))

In [14]:
hoge = []
hoge.append(wakati_sentences[0][0])
hoge += wakati_sentences[0][1]
hoge += wakati_sentences[0][2]
hoge

['既に', 'さ', 'ま', 'ざ', 'ま', 'な']

In [15]:
for i in enumerate(wakati_sentences):
    continue
print(i[0])

1711


In [16]:
wcount = 256

h_sentences = []
t_sentences = []

# wcount moji me kara kesu

for i in enumerate(wakati_sentences):
    #h_sent = []
    h_len = 0
    hn = 0
    #h_sent.append(wakati_sentences[i[0]][0])
    h_len += len(wakati_sentences[i[0]][0])
    while h_len < wcount:
        try:
            hn += 1
            if wakati_sentences[i[0]][hn]:
                #print(hn, wakati_sentences[i[0]][hn])
                #h_sent.append(wakati_sentences[i[0]][hn])
                h_len += len(wakati_sentences[i[0]][hn])
        except IndexError:
            break
    h_sentences.append(sentences[i[0]][:hn])
    
    #t_sent = []
    t_len = 0
    tn = 2
    #t_sent.append(wakati_sentences[i[0]][-2])
    t_len += len(wakati_sentences[i[0]][-2])
    while t_len < wcount:
        try:
            tn += 1
            if wakati_sentences[i[0]][tn]:
                #print(tn, wakati_sentences[i[0]][tn])
                #t_sent.append(wakati_sentences[i[0]][tn])
                t_len += len(wakati_sentences[i[0]][tn])
        except IndexError:
            break
    t_sentences.append(sentences[i[0]][-tn:])

In [17]:
h_sentences[0]

'既にさまざまなメディアで報じられているが東日本大震災後、結婚相談所を訪れる女性が急増したり、結婚指輪の売上が増加しているという。今まで結婚に興味がなかった独身者たちが転じて結婚に前向きな行動をとっているようだ。都内の企業に勤めるトモさん37歳は東日本大震災の時、帰宅困難になりましたよね。彼氏がいる同僚たちは彼氏と連絡'

In [18]:
hsentences = np.array(h_sentences)
tsentences = np.array(t_sentences)

In [19]:
ssentences = np.array(summaries)

In [20]:
ssentences

array(['思いがけないことがある人間は、未来を教えてくれることがある。',
       'アイドルファンは激戦時代のアイドルだ。アイドルそのものを支えるファンの態度も変わるのだろうか?',
       '自分は30年ぶりに夫と結婚しようと思った。', ..., '思いがけないクリスマスがもうすぐ到来!',
       '新型コロナウイルスによる感染症(COVID-19)が世界中で猛威を振るっている。',
       'クリスマスのあいさつは、とても大変だった。'], dtype=object)

In [21]:
np.array(ssentences).shape

(1712,)

In [22]:
emp = []
asentences = np.append(emp, copy.deepcopy(sentences))
ksentences = np.append(emp, copy.deepcopy(sentences))
kksentences = np.append(emp, copy.deepcopy(sentences))

# wcount moji me kara kesu

for i in enumerate(sentences):
    if len(i[1])>wcount:
        asentences[i[0]] = sentences[i[0]][:wcount]

# ushiro kara wcount moji toru

for i in enumerate(sentences):
    if len(i[1])>wcount:
        ksentences[i[0]] = sentences[i[0]][-wcount:]

# ushiro kara wcount moji toru ichiban ketsu wa toranai

for i in enumerate(sentences):
    if len(i[1])>wcount:
        am = wcount+10
        a = sentences[i[0]][-am:]
        kksentences[i[0]] = a[:wcount]

In [23]:
ksentences[2]

'健眠、健食を心がけ規則正しい生活をすること、それからストレスを溜めず発散すること。おケイコをはじめたりするのもいいだろうし、そんな余裕はないというなら、最寄り駅の2つ手前で降りてジョギングしながら帰るとか、できる範囲で充分だ。そして最後に、大豆製品に多く含まれるイソフラボンを摂ること。イソフラボンは女性ホルモンと同じような働きをすることは、知っている人も多いだろう。異性にはもちろん、同性にも見られたくないヒゲ！私も実はという独女世代のみなさん！たっぷり女性ホルモンを蓄えて、太いヒゲとさよならしよう。パンチ広'

# BERT Tokenizerを用いて単語分割・IDへ変換
## Tokenizerの準備
単語分割とIDへ変換

# テスト実行

In [24]:
# 最大単語数の確認
max_len = []
# 1文づつ処理
for sent in hsentences:
    # Tokenizeで分割
    h_token_words = tokenizer.tokenize(sent)
    # 文章数を取得してリストへ格納
    max_len.append(len(h_token_words))
for sent in tsentences:
    # Tokenizeで分割
    t_token_words = tokenizer.tokenize(sent)
    # 文章数を取得してリストへ格納
    max_len.append(len(t_token_words))
# 最大の値を確認
print('最大単語数: ', max(max_len))
print('上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数')

最大単語数:  117
上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数


In [25]:
h_input_ids = []
t_input_ids = []
a_input_ids = []
k_input_ids = []
kk_input_ids = []
s_input_ids = []
h_attention_masks = []
t_attention_masks = []
a_attention_masks = []
k_attention_masks = []
kk_attention_masks = []
s_attention_masks = []

# 1文づつ処理
for sent in hsentences:
    hencoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = wcount+2,  # I think maximum 文章の長さを固定（Padding/Trancatinating）
                        truncation=True,                
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )
    # print(hencoded_dict)

    h_input_ids.append(hencoded_dict['input_ids'])
    h_attention_masks.append(hencoded_dict['attention_mask'])
    
for sent in tsentences:
    tencoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = wcount+2,  # I think maximum 文章の長さを固定（Padding/Trancatinating）
                        truncation=True,                
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )
    # https://qiita.com/YuiKasuga/items/343309257da1798c1b63

    # 単語IDを取得    
    t_input_ids.append(tencoded_dict['input_ids'])

    # Attention　maskの取得
    t_attention_masks.append(tencoded_dict['attention_mask'])
    
for sent in asentences:
    aencoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = wcount+2,  # I think maximum 文章の長さを固定（Padding/Trancatinating）
                        truncation=True,                
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )
    # print(hencoded_dict)

    a_input_ids.append(aencoded_dict['input_ids'])
    a_attention_masks.append(aencoded_dict['attention_mask'])
    
for sent in ksentences:
    kencoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = wcount+2,  # I think maximum 文章の長さを固定（Padding/Trancatinating）
                        truncation=True,                
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )
    # print(hencoded_dict)

    k_input_ids.append(kencoded_dict['input_ids'])
    k_attention_masks.append(kencoded_dict['attention_mask'])
    
for sent in kksentences:
    kkencoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = wcount+2,  # I think maximum 文章の長さを固定（Padding/Trancatinating）
                        truncation=True,                
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )
    # print(hencoded_dict)

    kk_input_ids.append(kkencoded_dict['input_ids'])
    kk_attention_masks.append(kkencoded_dict['attention_mask'])
    
for summa in ssentences:
    sencoded_dict = tokenizer.encode_plus(
                        summa,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = wcount+2,  # I think maximum 文章の長さを固定（Padding/Trancatinating）
                        truncation=True,                
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )
    # 単語IDを取得    
    s_input_ids.append(sencoded_dict['input_ids'])
    # Attention　maskの取得
    s_attention_masks.append(sencoded_dict['attention_mask'])


In [26]:
# リストに入ったtensorを縦方向（dim=0）へ結合
h_input_ids = torch.cat(h_input_ids, dim=0)
t_input_ids = torch.cat(t_input_ids, dim=0)
a_input_ids = torch.cat(a_input_ids, dim=0)
k_input_ids = torch.cat(k_input_ids, dim=0)
kk_input_ids = torch.cat(kk_input_ids, dim=0)
s_input_ids = torch.cat(s_input_ids, dim=0)
h_attention_masks = torch.cat(h_attention_masks, dim=0)
t_attention_masks = torch.cat(t_attention_masks, dim=0)
a_attention_masks = torch.cat(a_attention_masks, dim=0)
k_attention_masks = torch.cat(k_attention_masks, dim=0)
kk_attention_masks = torch.cat(kk_attention_masks, dim=0)
s_attention_masks = torch.cat(s_attention_masks, dim=0)

# tenosor型に変換
labels = torch.tensor(labels)

In [27]:
# 確認
print('Original: ', hsentences[0])
print('Original: ', ssentences[0])
print('Token IDs:', s_input_ids[0])
print(type(labels), labels)
print('head')
print(torch.Tensor.size(h_input_ids))
print(torch.Tensor.size(h_attention_masks))
print('summary')
print(torch.Tensor.size(s_input_ids))
print(torch.Tensor.size(s_attention_masks))
print(torch.Tensor.size(labels))
print(ssentences.size)

Original:  既にさまざまなメディアで報じられているが東日本大震災後、結婚相談所を訪れる女性が急増したり、結婚指輪の売上が増加しているという。今まで結婚に興味がなかった独身者たちが転じて結婚に前向きな行動をとっているようだ。都内の企業に勤めるトモさん37歳は東日本大震災の時、帰宅困難になりましたよね。彼氏がいる同僚たちは彼氏と連絡
Original:  思いがけないことがある人間は、未来を教えてくれることがある。
Token IDs: tensor([    9,  2969,    12, 18143,  1272,   858,    11,     7,  3638, 15718,
        17379,  1272,     8,     2,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,    

In [28]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torchvision import transforms, datasets
from transformers import RobertaForMaskedLM

# データセットクラスの作成
# dataset = TensorDataset(h_input_ids, h_attention_masks, labels)
hdataset = TensorDataset(h_input_ids, h_attention_masks, labels)
tdataset = TensorDataset(t_input_ids, t_attention_masks, labels)
adataset = TensorDataset(a_input_ids, a_attention_masks, labels)
kdataset = TensorDataset(k_input_ids, k_attention_masks, labels)
kkdataset = TensorDataset(kk_input_ids, kk_attention_masks, labels)
sdataset = TensorDataset(s_input_ids, s_attention_masks, labels)
# dataset = CustomTensorDataset(tensors = (input_ids, labels), transform = None)


In [29]:
# 80%地点のIDを取得
num_dataset = len(hdataset)
#train_size = int(0.1 * num_dataset)
#val_size = num_dataset - train_size

train_size = 100
val_size = len(hdataset) - train_size

print('訓練データ数:{}'.format(train_size))
print('検証データ数:{}'.format(val_size))

# データローダーの作成
batch_size = 50

data_transform = transforms.Compose([
    synreplace(1),
    randinsert(3),
    randdelete(0.15), 
    randswap(2)
])


class MySubset(torch.utils.data.Dataset):
    def __init__(self, dataset, indices, transform=None):
        self.dataset = dataset
        self.indices = indices
        self.transform = transform

    def __getitem__(self, idx):
        xa, mask, label = self.dataset[self.indices[idx]]
        if self.transform:
            xa = self.transform(xa)
        return xa, mask, label

    def __len__(self):
        return len(self.indices)
    
indices = np.random.choice(num_dataset, num_dataset, replace=False)

# データセhttp://localhost:8888/notebooks/bert-zuco/augmentation/BERTclassification-mydata-local-augmentation-aug.ipynb#ットを分割
h_train_dataset, h_val_dataset = random_split(hdataset, [train_size, val_size])
t_train_dataset, t_val_dataset = random_split(tdataset, [train_size, val_size])
a_train_dataset, a_val_dataset = random_split(adataset, [train_size, val_size])
k_train_dataset, k_val_dataset = random_split(kdataset, [train_size, val_size])
kk_train_dataset, kk_val_dataset = random_split(kkdataset, [train_size, val_size])
s_train_dataset, s_val_dataset = random_split(sdataset, [train_size, val_size])

h_train_dataset = MySubset(hdataset, indices[:train_size], data_transform)
h_val_dataset = MySubset(hdataset, indices[train_size:])
t_train_dataset = MySubset(tdataset, indices[:train_size], data_transform)
t_val_dataset = MySubset(tdataset, indices[train_size:])
a_train_dataset = MySubset(adataset, indices[:train_size], data_transform)
a_val_dataset = MySubset(adataset, indices[train_size:])
k_train_dataset = MySubset(kdataset, indices[:train_size], data_transform)
k_val_dataset = MySubset(kdataset, indices[train_size:])
kk_train_dataset = MySubset(kkdataset, indices[:train_size], data_transform)
kk_val_dataset = MySubset(kkdataset, indices[train_size:])
s_train_dataset = MySubset(sdataset, indices[:train_size], data_transform)
s_val_dataset = MySubset(sdataset, indices[train_size:])


# 訓練データローダー
# shuffle True/False to compare or not
# wakati head
h_train_dataloader = DataLoader(
            h_train_dataset,  
#            sampler = RandomSampler(h_train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size,
            shuffle = True
            )
# wakati tail
t_train_dataloader = DataLoader(
            t_train_dataset,
#            sampler = RandomSampler(t_train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size,
            shuffle = True
            )
# letters atama
a_train_dataloader = DataLoader(
            a_train_dataset,
#            sampler = RandomSampler(a_train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size,
            shuffle = True
            )
#letters ketsu
k_train_dataloader = DataLoader(
            k_train_dataset,
#            sampler = RandomSampler(k_train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size,
            shuffle = True
            )
# letters ketsu -10
kk_train_dataloader = DataLoader(
            kk_train_dataset,
#            sampler = RandomSampler(kk_train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size,
            shuffle = True
            )
# summary
s_train_dataloader = DataLoader(
            s_train_dataset,
#            sampler = RandomSampler(s_train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size,
            shuffle = True
            )


# 検証データローダー
h_validation_dataloader = DataLoader(
            h_val_dataset, 
#            sampler = SequentialSampler(h_val_dataset), # 順番にデータを取得してバッチ化
            batch_size = 1,
            shuffle = False
        )
t_validation_dataloader = DataLoader(
            t_val_dataset, 
#            sampler = SequentialSampler(t_val_dataset), # 順番にデータを取得してバッチ化
            batch_size = 1,
            shuffle = False
        )
a_validation_dataloader = DataLoader(
            a_val_dataset, 
#            sampler = SequentialSampler(a_val_dataset), # 順番にデータを取得してバッチ化
            batch_size = 1,
            shuffle = False
        )
k_validation_dataloader = DataLoader(
            k_val_dataset, 
#            sampler = SequentialSampler(k_val_dataset), # 順番にデータを取得してバッチ化
            batch_size = 1,
            shuffle = False
        )
kk_validation_dataloader = DataLoader(
            kk_val_dataset, 
#            sampler = SequentialSampler(kk_val_dataset), # 順番にデータを取得してバッチ化
            batch_size = 1,
            shuffle = False
        )
s_validation_dataloader = DataLoader(
            s_val_dataset, 
#            sampler = SequentialSampler(s_val_dataset),
            batch_size = 1,
            shuffle = False
        )

訓練データ数:100
検証データ数:1612


In [30]:
from transformers import BertForSequenceClassification,AdamW,BertConfig

# BertForSequenceClassification 学習済みモデルのロード
model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pre trainedモデルの指定
    num_labels = 2, # ラベル数（今回はBinaryなので2、数値を増やせばマルチラベルも対応可）
    output_attentions = False, # アテンションベクトルを出力するか
    output_hidden_states = False, # 隠れ層を出力するか
).to(device)

# モデルをGPUへ転送
#model.cuda()


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [31]:
# 最適化手法の設定
optimizer = AdamW(model.parameters(), lr=2e-5)

In [32]:
alloutputs = []

def h_train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    for batch in h_train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = output.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
#                print(output)
#                print(output['logits'])
        alloutputs.append(output['logits'].to('cpu'))
    return train_loss, alloutputs

def h_validation(model):
    model.eval()# 訓練モードをオフ
    val_loss = 0
    alloutputs = []
    with torch.no_grad(): # 勾配を計算しない
        for batch in h_validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                (loss, logits) = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
            output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
            loss = output.loss
            val_loss += loss.item()
            alloutputs.append(output['logits'].to('cpu'))
    return val_loss, alloutputs

In [33]:
alloutputs = []

def t_train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    for batch in t_train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = output.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
#                print(output)
#                print(output['logits'])
        alloutputs.append(output['logits'].to('cpu'))
    return train_loss, alloutputs

def t_validation(model):
    model.eval()# 訓練モードをオフ
    val_loss = 0
    alloutputs = []
    with torch.no_grad(): # 勾配を計算しない
        for batch in t_validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                (loss, logits) = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
            output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
            loss = output.loss
            val_loss += loss.item()
            alloutputs.append(output['logits'].to('cpu'))
    return val_loss, alloutputs

In [34]:
alloutputs = []

def a_train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    for batch in a_train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = output.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
#                print(output)
#                print(output['logits'])
        alloutputs.append(output['logits'].to('cpu'))
    return train_loss, alloutputs

def a_validation(model):
    model.eval()# 訓練モードをオフ
    val_loss = 0
    alloutputs = []
    with torch.no_grad(): # 勾配を計算しない
        for batch in a_validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                (loss, logits) = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
            output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
            loss = output.loss
            val_loss += loss.item()
            alloutputs.append(output['logits'].to('cpu'))
    return val_loss, alloutputs

In [35]:
alloutputs = []

def k_train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    for batch in k_train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = output.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
#                print(output)
#                print(output['logits'])
        alloutputs.append(output['logits'].to('cpu'))
    return train_loss, alloutputs

def k_validation(model):
    model.eval()# 訓練モードをオフ
    val_loss = 0
    alloutputs = []
    with torch.no_grad(): # 勾配を計算しない
        for batch in k_validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                (loss, logits) = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
            output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
            loss = output.loss
            val_loss += loss.item()
            alloutputs.append(output['logits'].to('cpu'))
    return val_loss, alloutputs

In [36]:
alloutputs = []

def kk_train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    for batch in kk_train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = output.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
        alloutputs.append(output['logits'].to('cpu'))
    return train_loss, alloutputs

def kk_validation(model):
    model.eval()# 訓練モードをオフ
    val_loss = 0
    alloutputs = []
    with torch.no_grad(): # 勾配を計算しない
        for batch in kk_validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                (loss, logits) = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
            output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
            loss = output.loss
            val_loss += loss.item()
            alloutputs.append(output['logits'].to('cpu'))
    return val_loss, alloutputs

In [37]:
alloutputs = []

def s_train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    for batch in s_train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = output.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
#                print(output)
#                print(output['logits'])
        alloutputs.append(output['logits'].to('cpu'))
    return train_loss, alloutputs

def s_validation(model):
    model.eval()# 訓練モードをオフ
    val_loss = 0
    alloutputs = []
    with torch.no_grad(): # 勾配を計算しない
        for batch in s_validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                (loss, logits) = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
            output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
            loss = output.loss
            val_loss += loss.item()
            alloutputs.append(output['logits'].to('cpu'))
    return val_loss, alloutputs

In [38]:
# 学習の実行
max_epoch = 20
t_train_loss_ = []
t_test_loss_ = []
h_train_loss_ = []
h_test_loss_ = []
a_train_loss_ = []
a_test_loss_ = []
k_train_loss_ = []
k_test_loss_ = []
kk_train_loss_ = []
kk_test_loss_ = []
s_train_loss_ = []
s_test_loss_ = []

h_train_loss = 0
t_train_loss = 0
a_train_loss = 0
k_train_loss = 0
kk_train_loss = 0
s_train_loss = 0

In [39]:
for epoch in range(max_epoch):
    h_train_ = h_train(model)
    h_train_loss_.append(h_train_)
    t_train_ = t_train(model)
    t_train_loss_.append(t_train_)
    a_train_ = a_train(model)
    a_train_loss_.append(a_train_)
    k_train_ = k_train(model)
    k_train_loss_.append(k_train_)
    kk_train_ = kk_train(model)
    kk_train_loss_.append(kk_train_)
    s_train_ = s_train(model)
    s_train_loss_.append(s_train_)
    if epoch%10 == 0:
        print('epoch: ', epoch)

OutOfMemoryError: CUDA out of memory. Tried to allocate 38.00 MiB (GPU 0; 23.64 GiB total capacity; 7.12 GiB already allocated; 24.12 MiB free; 7.24 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
h_test_loss_ = h_validation(model)
t_test_loss_ = t_validation(model)
a_test_loss_ = a_validation(model)
k_test_loss_ = k_validation(model)
kk_test_loss_ = kk_validation(model)
s_test_loss_ = s_validation(model)

In [ ]:
sents = []

for i in range(df.shape[0]):
    sents.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[i])))

#print(sents)
sents = pd.DataFrame(sents)
print(type(sents))

In [ ]:
lsentences = np.append(emp, copy.deepcopy(sentences))

l_input_ids = []
l_attention_masks = []

for sent in lsentences:
    aencoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = wcount+2,  # I think maximum 文章の長さを固定（Padding/Trancatinating）
                        truncation=True,                
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )
    # print(hencoded_dict)

    l_input_ids.append(aencoded_dict['input_ids'])
    l_attention_masks.append(aencoded_dict['attention_mask'])

l_input_ids = torch.cat(l_input_ids, dim=0)
l_attention_masks = torch.cat(l_attention_masks, dim=0)
ldataset = TensorDataset(l_input_ids, l_attention_masks, labels)
l_train_dataset, l_val_dataset = random_split(ldataset, [train_size, val_size])

# type soroete X train test Y train test wo kaizan suru

In [ ]:
'''
X_train, X_test, y_train, y_test = train_test_split(
    sents, labels, test_size=0.95, random_state=42, shuffle=True
    )
print(y_train)

# x ga transformer id  y ga raberu
'''

In [ ]:
'''
print(type(y_train))
print(type(l_train_dataset[0][2]))
'''

In [ ]:
Xtrain = l_train_dataset[:][0] 
Xtest = l_val_dataset[:][0]
ytrain = l_train_dataset[:][2]
ytest = l_val_dataset[:][2]

X_train = Xtrain.to('cpu').detach().numpy().copy()
X_test = Xtest.to('cpu').detach().numpy().copy()
y_train = ytrain.to('cpu').detach().numpy().copy()
y_test = ytest.to('cpu').detach().numpy().copy()

In [ ]:
lgb_clf = lightgbm.LGBMClassifier(max_depth=50, num_iterations=30)
eval_res = {}
lgb_clf.fit(X_train, y_train, eval_set=[(X_test, y_test), (X_train, y_train)], verbose=5)

In [ ]:
l_preds = lgb_clf.predict(X_test)
'''
y_pred = lgb_clf.predict(X_test)
print(y_pred)
print(len(y_pred))
dnum = len(y_pred) % batch_size
print(dnum)
l_preds = y_pred[-dnum:]
'''

In [ ]:
print(len(l_preds))

In [ ]:
'''
y_pred = lgb_clf.predict(X_test)
print('head: ', statistics.mean(h_test_accuracy))
print('tail: ', statistics.mean(t_test_accuracy))
print('atama: ', statistics.mean(a_test_accuracy))
print('ketsu: ', statistics.mean(k_test_accuracy))
print('ketsu kestu: ', statistics.mean(kk_test_accuracy))
print('summary: ', statistics.mean(s_test_accuracy))
print('LGBM: ', accuracy_score(y_test, y_pred))
'''

In [ ]:
h_pred_ = []
t_pred_ = []
a_pred_ = []
k_pred_ = []
kk_pred_ = []
s_pred_ = []

for i in range(len(h_test_loss_[1])):
    h_pred_.append(np.argmax(np.array(h_test_loss_[1][i])))
    t_pred_.append(np.argmax(np.array(t_test_loss_[1][i])))
    a_pred_.append(np.argmax(np.array(a_test_loss_[1][i])))
    k_pred_.append(np.argmax(np.array(k_test_loss_[1][i])))
    kk_pred_.append(np.argmax(np.array(kk_test_loss_[1][i])))
    s_pred_.append(np.argmax(np.array(s_test_loss_[1][i])))
    

In [ ]:
vlabel = []
for a,b,c in h_validation_dataloader:
    vlabel.append(c.detach().numpy())

In [ ]:
h_pred_df = pd.DataFrame(h_pred_, columns=['h_pred_label'])
t_pred_df = pd.DataFrame(t_pred_, columns=['t_pred_label'])
a_pred_df = pd.DataFrame(a_pred_, columns=['a_pred_label'])
k_pred_df = pd.DataFrame(k_pred_, columns=['k_pred_label'])
kk_pred_df = pd.DataFrame(kk_pred_, columns=['kk_pred_label'])
s_pred_df = pd.DataFrame(s_pred_, columns=['s_pred_label'])
# l_preds_df = pd.DataFrame(l_preds, columns=['l_pred_label'])
label_df = pd.DataFrame(vlabel, columns=['true_label'])
accuracy_df = pd.concat([h_pred_df, t_pred_df, a_pred_df, k_pred_df, kk_pred_df, s_pred_df, label_df], axis=1)
accuracy_df.head(50)

In [ ]:
hpreds = h_pred_df.values
tpreds = t_pred_df.values
apreds = a_pred_df.values
kpreds = k_pred_df.values
kkpreds = kk_pred_df.values
spreds = s_pred_df.values
lpreds = np.array(l_preds)
preds = []
pred = 0
m = 7

for i in range(len(hpreds)):
    pred = hpreds[i]+tpreds[i]+apreds[i]+kpreds[i]+kkpreds[i]+spreds[i]+lpreds[i]
    if pred/m < 0.5:
        pred = 0
    else:
        pred = 1
    preds.append(pred)

In [ ]:
preds_df = pd.DataFrame(preds, columns=['pred_label'])
label_df = pd.DataFrame(vlabel, columns=['true_label'])
ensaccuracy_df = pd.concat([preds_df, label_df], axis=1)
ensaccuracy_df

# pred_label accuracy

In [ ]:
cor = 0
ypnum = 0 #yosoku
spnum = 0 #seikai
pnum = 0
rnum = 0
for i in range(len(preds_df)):
    if preds_df.values[i] == label_df.values[i]:
        cor += 1
    if preds_df.values[i] == 0:
        ypnum += 1
        if label_df.values[i] == 0:
            pnum += 1
    if label_df.values[i] == 0:
        spnum += 1
        if preds_df.values[i] == 0:
            rnum += 1
        
100*cor/len(preds_df)

# pred_label F1

In [ ]:
# tekigou
tp = pnum/ypnum
# saigen
sp = rnum/spnum

(tp*sp)/(tp+sp)

In [ ]:
def accuracy(pdf):
    cor = 0
    for i in range(len(preds_df)):
        if pdf[i] == label_df.values[i]:
            cor += 1
    acc = 100*cor/len(pdf)
    return acc

def fscore(pdf):
    cor = 0
    ypnum = 0 #yosoku
    spnum = 0 #seikai
    pnum = 0
    rnum = 0
    fone = 0
    for i in range(len(preds_df)):
        if pdf[i] == label_df.values[i]:
            cor += 1
        if pdf[i] == 0:
            ypnum += 1
            if label_df.values[i] == 0:
                pnum += 1
        if label_df.values[i] == 0:
            spnum += 1
            if pdf[i] == 0:
                rnum += 1
    # tekigou
    tp = pnum/ypnum
    # saigen
    sp = rnum/spnum
    # f1
    fone = (tp*sp)/(tp+sp)
    return fone

In [ ]:
print('head', accuracy(hpreds), fscore(hpreds))
print('tail', accuracy(tpreds), fscore(tpreds))
print('atama', accuracy(apreds), fscore(apreds))
print('ketsu', accuracy(kpreds), fscore(kpreds))
print('ketsu-10', accuracy(kkpreds), fscore(kkpreds))
print('summary', accuracy(spreds), fscore(spreds))
# print('lgbm', accuracy(lpreds), fscore(lpreds))
print('all', accuracy(preds_df.values), fscore(preds_df.values))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(h_test_loss_)), h_test_loss_, label="Head")
plt.plot(range(len(t_test_loss_)), t_test_loss_, label="Tail")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()  # ラベルがあるときは、きちんとplt.show()を呼び出すこと

In [ ]:
# 予測結果の確認
# print(f'出力:{h_preds}')

In [ ]:
'''
# 比較しやすい様にpd.dataframeへ整形
import numpy as np
# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1'])
## np.argmaxで大き方の値を取得
pred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label'])
label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label'])
accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)
accuracy_df.head()
'''

In [ ]:
'''
correct = 0
test_accuracy = 0

p_list = []
l_list = []

p_list = pred_df.values.tolist()
l_list = label_df.values.tolist()

#print(type(pred_df))

for i in range(len(b_labels)):
    if(p_list[i] == l_list[i]):
        correct += 1

test_accuracy = correct/len(b_labels)

print(test_accuracy)
'''

In [ ]:
len(h_train_loss_)

In [ ]:
H_train_loss = []
T_train_loss = []
A_train_loss = []
K_train_loss = []
KK_train_loss = []
S_train_loss = []

for i in range(max_epoch):
    H_train_loss.append(h_train_loss_[i][0])
    T_train_loss.append(t_train_loss_[i][0])
    A_train_loss.append(a_train_loss_[i][0])
    K_train_loss.append(k_train_loss_[i][0])
    KK_train_loss.append(kk_train_loss_[i][0])
    S_train_loss.append(s_train_loss_[i][0])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(H_train_loss)), H_train_loss, label="Head")
plt.plot(range(len(T_train_loss)), T_train_loss, label="Tail")
plt.plot(range(len(A_train_loss)), A_train_loss, label="WakatiHead")
plt.plot(range(len(K_train_loss)), K_train_loss, label="WakatiTail")
plt.plot(range(len(KK_train_loss)), KK_train_loss, label="WakatiTail-10")
plt.plot(range(len(S_train_loss)), S_train_loss, label="Summary")


plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()  # ラベルがあるときは、きちんとplt.show()を呼び出すこと